In [2]:
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision.utils as vutils
import random
import os
import shutil
import torch
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
import numpy as np
import os

In [17]:
# Set Defaults
num_epochs = 10 
batch_size = 16
num_classes = 10 # number of classes for dataset
lr = 0.0002 
b1 = 0.5 # adam: decay of first order momentum of gradient
b2 = 0.999 # adam: decay of first order momentum of gradient
n_cpu = 8 # number of cpu threads to use during batch generation
latent_dim = 100 # dimensionality of the latent space
img_size = 32 # size of each image dimension
channels = 1 # number of output image channels
sample_interval = 400 # interval between image sampling

In [4]:
# Set cuda
if torch.cuda.is_available():
    cuda = True 
else:
    cuda = False

In [5]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.zero_()

In [9]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.use_gpu = cuda
        drop_rate = 0.20
        num_classes = 2
        input_channels = 3
        ndf = 16
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=input_channels, out_channels=ndf, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(drop_rate),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=ndf, out_channels=ndf*2, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(drop_rate),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=ndf*2, out_channels=ndf*4, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(drop_rate),
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=ndf*4, out_channels=ndf*8, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf*8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(drop_rate),
        )
        
        self.main = nn.Sequential(
            nn.Conv2d(in_channels=input_channels, out_channels=ndf, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(drop_rate),
            nn.Conv2d(in_channels=ndf, out_channels=ndf*2, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(drop_rate),
            nn.Conv2d(in_channels=ndf*2, out_channels=ndf*4, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(drop_rate),
            nn.Conv2d(in_channels=ndf*4, out_channels=ndf*8, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf*8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(drop_rate),
        )
        
        # The height and width of downsampled image
        ds_size = img_size // 2**4
        
        # Output layer
        self.linear1 = nn.Sequential(
            nn.Linear(in_features=128*ds_size**2, out_features=1),
            nn.Sigmoid()
        )

    def forward(self, x):
#         if isinstance(x.data, torch.cuda.FloatTensor) and self.use_gpu:
#             out = nn.parallel.data_parallel(self.main, x, range(1))
#         else:
#             out = self.main(x)

        print("---------------------------------------")
        print("input size = ", x.size())
        x = self.conv1(x)
        print("Conv1 output = ", x.size())
        x = self.conv2(x)
        print("Conv2 output = ", x.size())
        x = self.conv3(x)
        print("Conv3 output = ", x.size())
        x = self.conv4(x)
        print("Conv4 output = ", x.size())
        
        # Flatten the features to give as input to the fc linear classification layer
        out = x.view(x.shape[0], -1)
        print("Out flatten shape = ", out.size())
        out = self.linear1(out)
        print("Linear output = ", out.size())
        print("---------------------------------------")
        
        return out

In [11]:
# Initialize loss and model
loss = nn.BCELoss()
model = Model()

if cuda:
    loss.cuda()
    model.cuda()

In [12]:
# Initialize weights
model.apply(weights_init_normal)

Model(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace)
    (3): Dropout2d(p=0.2)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace)
    (3): Dropout2d(p=0.2)
  )
  (conv3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace)
    (3): Dropout2d(p=0.2)
  )
  (conv4): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, a

In [15]:
# Load Data

In [16]:
# Start Training
mo
for epoch in range(num_epochs):
    for i, (imgs, labels) in enumerate(dataloader):
        